# Navigation

Project #1 for the Udacity Reinforcement Learning Nanodegree.  
Eduardo Peynetti
__________

## Introduction

For this project, we will train an agent to navigate (and collect bananas!) in a large, square world.  

<img src="images/banana.png" />

A reward of +1 is provided for collecting a yellow banana, and a reward of -1 is provided for collecting 
a blue banana.  Thus, the goal of the agent is to collect as many yellow bananas as possible while 
avoiding blue bananas.  

The state space has 37 dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction. Given this information, the agent has to learn how to best select actions.  Four discrete actions are available, corresponding to:
- **`0`** - move forward.
- **`1`** - move backward.
- **`2`** - turn left.
- **`3`** - turn right.

The task is episodic, and in order to solve the environment, the agent must get an average score of +13 
over 100 consecutive episodes.

## Model

The problem is solved by using a Deep-Q Network (DQN), and a Double Deep-Q Network (DDQN). This architecture is based on the paper: [Human-level control through deep reinfocement learning](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf).

The code for the agents can be found in [here](https://github.com/lalopey/drl/blob/master/drltools/agent/agent.py), under the DQNAgent and DDQNAgent classes.

### Q-Learning

Q-learning (or SarsaMax) is a temporal difference (TD) learning method. TD learning combines ideas from Monte Carlo Decision (MD) and Dynamic Programming (DP) methods. It samples from the environment and performs updates based on current estimates.

Q-learning attempts to sequentially improve an estimate of the state-value function for policy $\pi$, or *quality*, which is defined as the expected return starting from state s, taking action a, and thereafter following policy $\pi$:

$$Q_{\pi}(s,a) = \mathbb{E}[G_t |S_t=s, A_t=a]$$

where $G_t$ is the sum of all (discounted) future rewards $r_t$.

We start by initializing Q arbitrarily, and thereafter update the value Q(s,a) by first following policy $\pi$,  obtaining a reward $r_t$ and then estimating the rewards later obtained using our old estimate for Q:

$$Q^{new}_{\pi}(s,a) = Q^{old}_{\pi}(s,a) + \alpha (r_t + \gamma \max_a Q_{\pi}(s_{t+1}, a) - Q^{old}_{\pi}(s,a))$$

In here, $r_t + \gamma \max_a Q_{\pi}(s_{t+1}, a)$ is known as the TD target. $\alpha$ is the learning rate and $\gamma$ is a discount factor. In Q-learning, we update the TD target by choosing the action that maximizes $Q_{\pi}(s_{t+1}, a)$, but other methods can choose different functions of Q, such as the average.


### Deep Q-Learning

Deep Q-Learning attempts to approximate the Q-function with a neural network. We try to minimize the mean squared error: 

$$J(\bf{w}) = \mathbb{E}\big[\big(Q_{\pi}(s,a)- \hat Q_{\pi}(s,a,\bf{w})\big)^2\big]$$

Where $\hat Q_{\pi}(s,a,\bf{w})$ is a neural network with parameters **w**. Taking the gradient with respect to **w**, we get:

$$\nabla_w J(\bf{w}) = -2\big(Q_{\pi}(s,a)- \hat Q(s,a,\bf{w})\big)\nabla_w \hat Q(s,a,\bf{w})$$

Taking a gradient descent update $\Delta\bf{w} = -\frac{\alpha}{2} \nabla_w \hat Q(s,a,\bf{w})$, and estimating the Q-function through Q-learning, we get:

$$\Delta\bf{w}=\alpha (r_t + \gamma \max_a \hat Q(s_{t+1},a,\bf{w}) - \hat Q(s_{t},a,\bf{w}))$$


Historically, attempting to approximate the function using neural networks has led to unstable or divergent results. The source of the instability comes from the correlations present in the sequence of observations, the fact that small updates to Q may significantly change the actions taken, and the correlations between Q and the target values.

The paper quoted above attempted to solve these issues using the following techniques:

#### Experience Replay

When the agent interacts with the environment, the sequence of observations can be highly correlated. The naive Q-learning algorithm that learns from each of these experience tuples in sequential order runs the risk of getting swayed by the effects of this correlation. By instead keeping track of a replay buffer and later sampling from this buffer at random, a method known as experience replay, we can prevent action values from oscillating or diverging catastrophically. Experience replay also allows us to learn more from individual observations multiple times, recall rare events, and in general make better use of our experience.

#### Fixed Q-targets

In Q-Learning, we update a guess with a guess, and this can potentially lead to harmful correlations. The TD update is done with the same function we are estimating. To avoid correlation issues, we can do the TD update with a different network whose weights are not updated during the learning step. 

$$\Delta\bf{w}=\alpha (r_t + \gamma \max_a \hat Q(s_{t+1},a,\bf{w^{-}}) - \hat Q(s_{t},a,\bf{w}))$$

The network used to calculate the TD target is known as the **target** network, while the network that we are updating is known as the **local** network.

After we update the local network, we an now update the target network using a linear approximation of the new weights in the local network and the current weights in the target network:

$$\bf{w}^{-} = \tau*\bf{w} + (1 - \tau)*w^{-}$$

#### Double DQN

The TD update in Q-learning takes the maximum of the Q-function over all possible actions. This can lead us to overestimate the importance of some action/value pairs, particularly early on when we don't have a good estimate of the Q function.

One way to solvent this issue is to look at the maximum of the Q-function over all actions in the following manner:

$$ \max_a \hat Q(s_{t+1},a,\bf{w}) = \hat Q(s_{t+1},argmax_a Q(s_{t+1},a,\bf{w}), \bf{w}))$$

So now, one can choose the best action using one network with weights **w**, and then evaluating that action with a different network with weights **w'**

$$ \max_a \hat Q(s_{t+1},a,\bf{w}) = \hat Q(s_{t+1},argmax_a Q(s_{t+1},a,\bf{w}), \bf{w'}))$$

This will reduce the chance that we overestimate certain value-action pairs. In our case, using $\bf{w'} = \bf{w^{-}}$ from the target network has been shown to work well in practice.


## Results

### Agent Training and Model Architecture

Udacity has provided us with a fully observable environment. The state space has 37 dimensions, 2 corresponding to the agent's velocity (left-right and front-back) and 35 to ray-based perception of objects around agent's forward direction. These can be broken down in 7 different angles (20, 45, 70, 90, 110, 135, 160), and each of these in 5 dimensions representing which object it finds and the distance to that object (yellow banana, wall, blue banana, agent, distance). The action space has 4 directions: left, right, forward, and backward.

The model architecture used to solve the problem is as follows:

- A target and local neural network, both with the same architecture, which consists of an input layer, two layers with 64 nodes each with ReLU activations, and a final layer mapping into the action space. A batch size of 64 is used.
- The local network is optimized with Adam, with a 5e-4 learning rate. The $\gamma$ in the TD update is 0.99
- The weights of the target network are updated through a soft update θ_target = τ*θ_local + (1 - τ)*θ_target with τ = 1e-3 
- A buffer size of 1e5 for experience replay. The buffer is sampled once every 4 steps. 
- The agent follows an epislon-greedy policy, with an initial $\epsilon = 1$, a decay at every step of 0.995, and a minimum $\epsilon$ of 0.01.

### DQN

The environment solved with DQN is solved in 499 episodes.

Episode 100	Average Score: 1.26  
Episode 200	Average Score: 4.60  
Episode 300	Average Score: 7.86  
Episode 400	Average Score: 10.62  
Episode 499	Average Score: 13.01	Episode score (max over agents): 16.00  
Environment solved in 499 episodes with an Average Score of 13.01  

<img src="images/dqn.png" />

### DDQN

The environment with DDQN is solved in 508 episodes, slightly more than DQN. It seemed DDQN took longer to improve early on than DQN but made up for it in the later stages. DDQN also had a higher maximum score episode (26 vs 21)

Episode 100	Average Score: 0.52  
Episode 200	Average Score: 3.47  
Episode 300	Average Score: 7.47  
Episode 400	Average Score: 10.03  
Episode 500	Average Score: 12.84  
Episode 508	Average Score: 13.07	Episode score (max over agents): 20.00  
Environment solved in 508 episodes with an Average Score of 13.07  

<img src="images/ddqn.png" />

# Future improvements


The DQN architecture can still be improved significantly. Some possible ideas are:

- [Dueling DQNs](https://arxiv.org/abs/1511.06581)
- [Learning from multi-step bootstrap agents](https://arxiv.org/abs/1602.01783)
- [Distributional DQNs](https://arxiv.org/abs/1707.06887)
- [Noisy DQNs](https://arxiv.org/abs/1706.10295)


